In [11]:
from copy import deepcopy
import time
import heapq
import numpy as np

In [129]:
Map

['##########',
 '#    3P  #',
 '#       A#',
 '## #### ##',
 '#   P P  #',
 '#        #',
 '##########']

In [12]:
def open_map(filename):
    file = open(filename)
    Map = file.read().split('\n')
    del Map[-1]
    # state = [
    #     [2,4], #Ambulance
    #     [[1, 8], [3, 2], [3, 9], [4,4]], #Patients
    #     {(1, 6): 1, (5, 2): 2, (5, 9): 1} #Hospitals
    # ] Sample State
    state = [[], [], {}]
    for i, row in enumerate(Map):
        for j, cell in enumerate(row):
            if cell == 'A':
                state[0] = [i, j]
            elif cell == 'P':
                state[1].append([i, j])
            elif cell.isdigit():
                state[2][i,j] = int(cell)
    start_state = state
    rawMap = deepcopy(Map)
    for i in range(len(rawMap)):
        rawMap[i] = rawMap[i].replace('1', ' ')
        rawMap[i] = rawMap[i].replace('2', ' ')
        rawMap[i] = rawMap[i].replace('3', ' ')
        rawMap[i] = rawMap[i].replace('P', ' ')
        rawMap[i] = rawMap[i].replace('A', ' ')
    return Map, rawMap, start_state

In [13]:
def printState(state):
    m = deepcopy(rawMap)
    a = state[0]
    m[a[0]] = m[a[0]][:a[1]] + 'A' + m[a[0]][a[1]+1:]
    for p in state[1]:
        m[p[0]] = m[p[0]][:p[1]] + 'P' + m[p[0]][p[1]+1:]
    for h in state[2]:
        m[h[0]] = m[h[0]][:h[1]] + str(state[2][h]) + m[h[0]][h[1]+1:]
    for row in m:
        print(row)
    print("---")


In [14]:
def print_steps(state, to_visit):
    s = state
    while s is not None:
        if print_steps:
            printState(s)
        s = to_visit[str(s)][0]


In [176]:
def Manhatan(state, cum_cost):
    cost = 0
    for p in state[1]:
        cost += np.min(np.sum(np.abs(np.array(list(state[2]))  - np.array(p)), axis=1))
    return cost + cum_cost + np.random.uniform()/100
def Euclidean(state, cum_cost):
    cost = 0
    for p in state[1]:
        cost += np.min(np.sum(np.power(np.array(list(state[2]))  - np.array(p), 2), axis=1))
    return np.sqrt(cost) + cum_cost + np.random.uniform()/100
def Patients(state, cum_cost):
    return len(state[2]) + cum_cost + np.random.uniform()/100

In [177]:
def Heuristic(start_state, h_func):
    visited, queue, to_visit = set(), [[h_func(start_state, 0), start_state]], dict()
    found = False
    unique_state = 0
    total_state = 1
    better_way = set()
    to_visit[str(start_state)] = [None, 0]
    while queue:
        cum_cost, state = heapq.heappop(queue)
#         print(cost, state)
        unique_state += 1
        if str(state) not in visited or str(state) in better_way:
            if str(state) in better_way:
                better_way.remove(str(state))
            visited.add(str(state))
            i,j = state[0]
            if (len(state[1]) == 0):
                found = True
                break
            for di, dj in [[-1, 0], [1, 0], [0, -1], [0, 1]]:
                new_state = deepcopy(state)
    #             new_state.parent = state
                ni = i+di; nj = j+dj
                n2i = i+2*di; n2j = j+2*dj
                if Map[ni][nj] != '#': # if neighbour is not wall
                    if [ni, nj] in state[1]: # if neighbour is a patient
                        if (n2i, n2j) in new_state[2] and new_state[2][n2i, n2j] > 0: #if 2neighbour is an hospital
                            new_state[0][0] += di
                            new_state[0][1] += dj
                            new_state[1].remove([ni, nj]) # remove patient
                            new_state[2][n2i, n2j] -= 1 # decrease hospital
                        elif ([n2i, n2j] not in new_state[1]) and  Map[n2i][n2j] != '#':
                            new_state[0][0] += di
                            new_state[0][1] += dj
                            new_state[1].remove([ni, nj]) # remove patient
                            new_state[1].append([n2i, n2j])
                            new_state[1].sort()
                        else:
                            continue
                    else:
                        new_state[0][0] += di
                        new_state[0][1] += dj
                    total_state += 1
                    if str(new_state) not in to_visit:
                        steps = to_visit[str(state)][1] + 1
                        heapq.heappush(queue, [h_func(new_state, cum_cost), new_state])
                        to_visit[str(new_state)] = [state, steps]
                    elif to_visit[str(state)][1] + 1 < to_visit[str(new_state)][1]:
                        heapq.heappush(queue, [h_func(new_state, cum_cost), new_state])
                        to_visit[str(new_state)] = [state, to_visit[str(state)][1] + 1]
                        better_way.add(str(new_state))

    return total_state, unique_state, to_visit, state, found

In [181]:
Map, rawMap, start_state = open_map("test3.txt")
tic = time.time()
ts, us, to_visit, goal_state, found = Heuristic(start_state, Manhatan)
print(time.time() - tic)
print(ts, us, found)
print("Steps: ", to_visit[str(goal_state)][1])
tic = time.time()
ts, us, to_visit, goal_state, found = Heuristic(start_state, Patients)
print(time.time() - tic)
print(ts, us, found)
print("Steps: ", to_visit[str(goal_state)][1])
tic = time.time()
ts, us, to_visit, goal_state, found = Heuristic(start_state, Euclidean)
print(time.time() - tic)
print(ts, us, found)
print("Steps: ", to_visit[str(goal_state)][1])

0.8475503921508789
13061 4795 True
Steps:  41
3.48002028465271
79792 29043 True
Steps:  39
2.191082715988159
29884 10928 True
Steps:  39


In [182]:
Map, rawMap, start_state = open_map("test2.txt")
tic = time.time()
ts, us, to_visit, goal_state, found = Heuristic(start_state, Manhatan)
print(time.time() - tic)
print(ts, us, found)
print("Steps: ", to_visit[str(goal_state)][1])
tic = time.time()
ts, us, to_visit, goal_state, found = Heuristic(start_state, Patients)
print(time.time() - tic)
print(ts, us, found)
print("Steps: ", to_visit[str(goal_state)][1])
tic = time.time()
ts, us, to_visit, goal_state, found = Heuristic(start_state, Euclidean)
print(time.time() - tic)
print(ts, us, found)
print("Steps: ", to_visit[str(goal_state)][1])

0.821213960647583
10772 4243 True
Steps:  27
1.3009214401245117
23042 9226 True
Steps:  27
1.3139302730560303
14408 5716 True
Steps:  27


In [183]:
Map, rawMap, start_state = open_map("test1.txt")
tic = time.time()
ts, us, to_visit, goal_state, found = Heuristic(start_state, Manhatan)
print(time.time() - tic)
print(ts, us, found)
print("Steps: ", to_visit[str(goal_state)][1])
tic = time.time()
ts, us, to_visit, goal_state, found = Heuristic(start_state, Patients)
print(time.time() - tic)
print(ts, us, found)
print("Steps: ", to_visit[str(goal_state)][1])
tic = time.time()
ts, us, to_visit, goal_state, found = Heuristic(start_state, Euclidean)
print(time.time() - tic)
print(ts, us, found)
print("Steps: ", to_visit[str(goal_state)][1])

0.025394678115844727
218 78 True
Steps:  11
0.043848514556884766
1302 469 True
Steps:  11
0.0202181339263916
313 111 True
Steps:  11
